In [2]:
import numpy as np
import time
from numpy.random import uniform

## List approach (so slow)

In [155]:
uncertainty_dict = {}
uncertainty_list = []
x_range = np.arange(-5,5.1,0.1)
y_range = np.arange(-5,5.1,0.1)
z_range = np.arange(0,5.1,0.1)

N = len(x_range)
for i in range(len(x_range)):
    for j in range(len(y_range)):
        for k in range(len(z_range)):
            element = (x_range[i], y_range[j], z_range[k])
            uncertainty_dict[element] = 1
            uncertainty_list.append([x_range[i], y_range[j], z_range[k]])
        
uncertainty_grids = np.array(uncertainty_list)
uncertainty_values = np.ones((uncertainty_grids.shape[0], ))

In [44]:
dist = 1e6
eps = 0.25
current_pos = [1.0, 2.0, 3.5]
closest_grid = None

for key, value in uncertainty_list.items():
    current_dist = np.sqrt((key[0]-current_pos[0])**2 + (key[1]-current_pos[1])**2 + (key[2]-current_pos[2])**2)
    if current_dist < dist:
        dist = current_dist
        closest_grid = key
        
    if current_dist <= eps:
        print ("that's right")
        break
        
closest_grid

that's right


(1.0, 2.0, 3.5)

## Array approach (much faster)

In [9]:
x_lim, y_lim, z_lim = 20, 20, 6
eps = 0.1
res = 1
X,Y,Z = np.mgrid[-x_lim:x_lim+eps:res, -y_lim:y_lim+eps:res, 0:z_lim+eps:res*2]
uncertainty_grids = np.vstack((X.flatten(), Y.flatten(), Z.flatten())).T
uncertainty_values = np.ones((uncertainty_grids.shape[0], ))
grid_visits = np.zeros((uncertainty_grids.shape[0], ))

print (uncertainty_grids.shape)
print (np.sqrt(uncertainty_grids.shape[0]))

(6724, 3)
82.0


In [8]:
uncertainty_grids[0]

array([-20., -20.,   0.])

In [11]:
current_pos = [1.75, 2.0, 3.5]

differences = current_pos-uncertainty_grids
distances = np.sum(differences*differences,axis=1)
min_ind = np.argmin(distances)
print("Closest grid: ",uncertainty_grids[min_ind])
print("Uncertainty value: ", uncertainty_values[min_ind])

# Closest N grids
dist = 5.0
indices = np.where(distances<dist)[0]
print ("Closest N Grids: ", len(indices))

Closest grid:  [2. 2. 4.]
Uncertainty value:  1.0
Closest N Grids:  25


In [72]:
def closest_grid(current_pos):
    differences = current_pos-uncertainty_grids
    distances = np.sum(differences*differences,axis=1)
    min_ind = np.argmin(distances)
    
    return min_ind

In [12]:
uncertainty_values[distances<dist] = 0.3

grid_visits[distances<dist] += 1

In [57]:
np.clip(np.exp(-grid_visits[distances<dist]), 1e-6, 1.0)

array([0.36787944, 0.36787944, 0.36787944, 0.36787944, 0.36787944,
       0.36787944, 0.36787944, 0.36787944, 0.36787944, 0.36787944,
       0.36787944, 0.36787944, 0.36787944, 0.36787944, 0.36787944,
       0.36787944, 0.36787944, 0.36787944, 0.36787944, 0.36787944,
       0.36787944, 0.36787944, 0.36787944, 0.36787944, 0.36787944,
       0.36787944, 0.36787944, 0.36787944, 0.36787944, 0.36787944,
       0.36787944, 0.36787944, 0.36787944, 0.36787944, 0.36787944,
       0.36787944, 0.36787944, 0.36787944, 0.36787944, 0.36787944,
       0.36787944, 0.36787944, 0.36787944, 0.36787944, 0.36787944,
       0.36787944, 0.36787944, 0.36787944, 0.36787944, 0.36787944])

In [89]:
drone_pos_stack = np.zeros(uncertainty_values.shape)
drone_pos_stack[min_ind] = 1
drone_pos_stack = drone_pos_stack.reshape(82,82)

grid_xyz_0 = np.array([uniform(-10, 10), uniform(-10, 10), uniform(0.5,3)])
grid_xyz_1 = grid_xyz_0 + np.array([uniform(0, 5), uniform(0, 5), uniform(0.5,3)])

print (grid_xyz_0)
print (grid_xyz_1)

print (closest_grid(grid_xyz_0))
print (closest_grid(grid_xyz_1))

[ 9.89994975 -6.88315257  2.92694057]
[12.84574727 -2.41378293  4.2853012 ]
4973
5486


In [90]:
x_range = np.arange(grid_xyz_0[0], grid_xyz_1[0], 0.25)
y_range = np.arange(grid_xyz_0[1], grid_xyz_1[1], 0.25)
z_range = np.arange(grid_xyz_0[2], grid_xyz_1[2], 0.5)

indices = []
for x in x_range:
    for y in y_range:
        for z in z_range:
            current_pos = np.array([x,y,z])
            current_ind = closest_grid(current_pos)
            if current_ind not in indices:
                indices.append(current_ind)

In [112]:
a = np.array([-1,2,-3])
np.zeros(3)

array([0., 0., 0.])

In [113]:
for i,j in zip(3,4):
    print (i+j)

TypeError: zip argument #1 must support iteration

In [109]:
np.min(uncertainty_grids[indices],axis=0)

array([10., -7.,  2.])

In [108]:
np.max(uncertainty_grids[indices],axis=0)

array([13., -3.,  4.])

In [13]:
obstacle_indices = [1576, 1577, 1578, 1579, 1580, 1581, 1582, 1583, 1584, 1585, 1586, 1587, 1588, 1589, 1590, 1591, 1592, 1593, 1594, 1595, 1596, 1597, 1598, 1599, 1600, 1601, 1602, 1603, 1604, 1605, 1606, 1607, 1608, 1609, 1610, 1611, 1612, 1613, 1614, 1615, 1616, 1617, 1618, 1619, 1620, 1621, 1622, 1623, 1624, 1625, 1626, 1627, 1740, 1741, 1742, 1743, 1744, 1745, 1746, 1747, 1748, 1749, 1750, 1751, 1752, 1753, 1754, 1755, 1756, 1757, 1758, 1759, 1760, 1761, 1762, 1763, 1764, 1765, 1766, 1767, 1768, 1769, 1770, 1771, 1772, 1773, 1774, 1775, 1776, 1777, 1778, 1779, 1780, 1781, 1782, 1783, 1784, 1785, 1786, 1787, 1788, 1789, 1790, 1791, 1904, 1905, 1906, 1907, 1908, 1909, 1910, 1911, 1912, 1913, 1914, 1915, 1916, 1917, 1918, 1919, 1920, 1921, 1922, 1923, 1924, 1925, 1926, 1927, 1928, 1929, 1930, 1931, 1932, 1933, 1934, 1935, 1936, 1937, 1938, 1939, 1940, 1941, 1942, 1943, 1944, 1945, 1946, 1947, 1948, 1949, 1950, 1951, 1952, 1953, 1954, 1955, 4140, 4141, 4142, 4143, 4144, 4145, 4146, 4147, 4148, 4149, 4150, 4151, 4152, 4153, 4154, 4155, 4156, 4157, 4158, 4159, 4160, 4161, 4162, 4163, 4164, 4165, 4166, 4167, 4168, 4169, 4170, 4171, 4172, 4173, 4174, 4175, 4176, 4177, 4178, 4179, 4180, 4181, 4182, 4183, 4184, 4185, 4186, 4187, 4188, 4189, 4190, 4191, 4192, 4193, 4194, 4195, 4196, 4197, 4198, 4199, 4200, 4201, 4202, 4203, 4204, 4205, 4206, 4207, 4208, 4209, 4210, 4211, 4212, 4213, 4214, 4215, 4216, 4217, 4218, 4219, 4220, 4221, 4222, 4223, 4224, 4225, 4226, 4227, 4304, 4305, 4306, 4307, 4308, 4309, 4310, 4311, 4312, 4313, 4314, 4315, 4316, 4317, 4318, 4319, 4320, 4321, 4322, 4323, 4324, 4325, 4326, 4327, 4328, 4329, 4330, 4331, 4332, 4333, 4334, 4335, 4336, 4337, 4338, 4339, 4340, 4341, 4342, 4343, 4344, 4345, 4346, 4347, 4348, 4349, 4350, 4351, 4352, 4353, 4354, 4355, 4356, 4357, 4358, 4359, 4360, 4361, 4362, 4363, 4364, 4365, 4366, 4367, 4368, 4369, 4370, 4371]


In [14]:
uncertainty_grids[obstacle_indices]

array([[-11.,   5.,   0.],
       [-11.,   5.,   2.],
       [-11.,   5.,   4.],
       [-11.,   5.,   6.],
       [-11.,   6.,   0.],
       [-11.,   6.,   2.],
       [-11.,   6.,   4.],
       [-11.,   6.,   6.],
       [-11.,   7.,   0.],
       [-11.,   7.,   2.],
       [-11.,   7.,   4.],
       [-11.,   7.,   6.],
       [-11.,   8.,   0.],
       [-11.,   8.,   2.],
       [-11.,   8.,   4.],
       [-11.,   8.,   6.],
       [-11.,   9.,   0.],
       [-11.,   9.,   2.],
       [-11.,   9.,   4.],
       [-11.,   9.,   6.],
       [-11.,  10.,   0.],
       [-11.,  10.,   2.],
       [-11.,  10.,   4.],
       [-11.,  10.,   6.],
       [-11.,  11.,   0.],
       [-11.,  11.,   2.],
       [-11.,  11.,   4.],
       [-11.,  11.,   6.],
       [-11.,  12.,   0.],
       [-11.,  12.,   2.],
       [-11.,  12.,   4.],
       [-11.,  12.,   6.],
       [-11.,  13.,   0.],
       [-11.,  13.,   2.],
       [-11.,  13.,   4.],
       [-11.,  13.,   6.],
       [-11.,  14.,   0.],
 